In [1]:
import wandb
from tqdm import tqdm
import numpy as np

api = wandb.Api()

def load_empirical_fixed_cost_runs(acq, tfn):
    runs = api.runs(path="ziv-scully-group/PandoraBayesOPT", filters={
        "sweep": "082ltdi0", 
        "config.problem": tfn,
        "config.policy": acq})
    
    configs_and_metrics = []
    for run in tqdm(runs):
        metric_keys = ["cumulative cost","best observed"]
        history = run.scan_history(keys = metric_keys, page_size=1_000_000_000)
        metrics = {k: [d[k] for d in history] for k in metric_keys}
        configs_and_metrics.append((run.config, metrics))

    return configs_and_metrics

In [2]:
acquisition_functions = {
    'RandomSearch': 'RandomSearch',
    'ExpectedImprovement':'ExpectedImprovement',
    'ThompsonSampling':'ThompsonSampling', 
    'KnowledgeGradient':'KnowledgeGradient',
    'MultiStepLookaheadEI':'MultiStepLookaheadEI',
    # 'Gittins_Lambda_01':'Gittins_Lambda01',
    # 'Gittins_Lambda_001':'Gittins_Lambda001',
    'Gittins_Lambda_0001':'Gittins_Lambda0001',
    'Gittins_Step_Divide2':'Gittins_Step_Divide2',
    # 'Gittins_Step_Divide5':'Gittins_Step_Divide5',
    # 'Gittins_Step_Divide10':'Gittins_Step_Divide10',
    # 'Gittins_Step_EIpu':'Gittins_Step_EIPC',
    }
target_functions = ["LunarLander"]

In [3]:
grouped_runs = {(a,t): load_empirical_fixed_cost_runs(a,t) for a in acquisition_functions.keys() for t in target_functions}

100%|██████████| 16/16 [00:12<00:00,  1.24it/s]


In [7]:
for a in acquisition_functions.keys():
    for t in target_functions:
        config_and_metrics_per_seed = grouped_runs[a,t]

        cumulative_cost_per_seed = np.array([m['cumulative cost'] for (_,m) in config_and_metrics_per_seed]).T
        best_observed_per_seed = np.array([m['best observed'] for (_,m) in config_and_metrics_per_seed]).T

        print(a, t, best_observed_per_seed.shape)
        
        best_25 = np.quantile(best_observed_per_seed, 0.25, axis=1)
        best_50 = np.quantile(best_observed_per_seed, 0.5, axis=1)
        best_75 = np.quantile(best_observed_per_seed, 0.75, axis=1)

        output = np.stack((cumulative_cost_per_seed.mean(axis=1), best_25, best_50, best_75),axis=-1)

        np.savetxt(f"results/quartiles/empirical/Empirical_{t}_{acquisition_functions[a]}.csv", output, header="cc, q25, q50, q75", delimiter=', ', comments='')

RandomSearch LunarLander (481, 16)
ExpectedImprovement LunarLander (481, 16)
ThompsonSampling LunarLander (481, 16)
Gittins_Lambda_01 LunarLander (481, 16)
Gittins_Lambda_001 LunarLander (481, 16)
Gittins_Lambda_0001 LunarLander (481, 16)
Gittins_Step_Divide2 LunarLander (481, 16)
Gittins_Step_Divide5 LunarLander (481, 16)
Gittins_Step_Divide10 LunarLander (481, 16)
Gittins_Step_EIpu LunarLander (481, 16)


In [4]:
for a in acquisition_functions.keys():
    for t in target_functions:
        config_and_metrics_per_seed = grouped_runs[a,t]

        cumulative_cost_per_seed = np.array([m['cumulative cost'] for (_,m) in config_and_metrics_per_seed if len(m['cumulative cost'])>0]).T 
        best_observed_per_seed = np.array([m['best observed'] for (_,m) in config_and_metrics_per_seed if len(m['best observed'])>0]).T
            
        print(a, t, best_observed_per_seed.shape)

        mean = np.mean(best_observed_per_seed, axis=1)
        se = np.std(best_observed_per_seed, axis=1, ddof=1) / np.sqrt(best_observed_per_seed.shape[1])
        
        mean_plus_2se = mean + 2 * se
        mean_minus_2se = mean - 2 * se

        output = np.stack((cumulative_cost_per_seed.mean(axis=1), mean_minus_2se, mean, mean_plus_2se),axis=-1)

        np.savetxt(f"results/SE/empirical/Empirical_{t}_{acquisition_functions[a]}.csv", output, header="cc, mean-2se, mean, mean+2se", delimiter=', ', comments='')

RandomSearch LunarLander (200, 16)
ExpectedImprovement LunarLander (200, 16)
ThompsonSampling LunarLander (200, 16)
KnowledgeGradient LunarLander (200, 16)
MultiStepLookaheadEI LunarLander (200, 16)
Gittins_Lambda_0001 LunarLander (200, 16)
Gittins_Step_Divide2 LunarLander (200, 16)
